In [1]:
import torch
from torch import nn
import numpy as np 
from math import sqrt

In [367]:
input_size = 16
batch_size = 13
target_size = 17
hidden_size = 8
n_head = 2
single_dim = hidden_size // n_head

In [420]:
input_size = 16
batch_size = 3
target_size = 16
hidden_size = 16
n_head = 2
single_dim = hidden_size // n_head

In [424]:
query = torch.FloatTensor(batch_size, input_size).uniform_(0, 1)
target = torch.FloatTensor(batch_size, target_size, input_size).uniform_(0, 1)
mask = torch.BoolTensor(batch_size, target_size).random_(0, 1)

In [454]:
W_q = nn.Linear(input_size, hidden_size, bias=False)
W_k = nn.Linear(input_size, hidden_size, bias=False)
W_v = nn.Linear(input_size, hidden_size, bias=False)
W_out = nn.Linear(hidden_size, input_size, bias=False)

In [455]:
q_c = W_q(query).reshape(batch_size, n_head, single_dim)
k = W_k(target).reshape(batch_size, target_size, n_head, single_dim).permute(0, 2, 1, 3)
v = W_v(target).reshape(batch_size, target_size, n_head, single_dim).permute(0, 2, 1, 3)
#both to be batch_size x n_head x target_size x single_dim

In [456]:
q_c.shape, k.shape, v.shape

(torch.Size([3, 2, 8]), torch.Size([3, 2, 16, 8]), torch.Size([3, 2, 16, 8]))

In [457]:
qk = torch.einsum("ijl,ijkl->ijk", [q_c,k]) / sqrt(single_dim)
# batch_size x n_head x target_size
softmaxed = qk.softmax(-1)

In [458]:
qk.shape

torch.Size([3, 2, 16])

In [459]:
_mask = mask.unsqueeze(1).repeat(1,2,1)

In [460]:
qk[_mask] = -100000.0

In [461]:
qk[0]

tensor([[ 0.0268, -0.0399,  0.0150,  0.0046,  0.0929,  0.0100,  0.0119, -0.0471,
          0.0187, -0.0111, -0.0330, -0.0608, -0.0294,  0.0429,  0.1020, -0.0361],
        [ 0.0341,  0.0854,  0.0263,  0.0335,  0.0392,  0.0930,  0.0598,  0.0103,
          0.0276,  0.0595, -0.0039,  0.0136,  0.0196,  0.0487,  0.0477,  0.0794]],
       grad_fn=<SelectBackward>)

In [462]:
softmaxed = qk.softmax(-1)

In [533]:
softmaxed.shape

torch.Size([3, 2, 16])

In [517]:
print(softmaxed.shape, v.shape)
nh = torch.einsum("ijk,ijkl->ijl", softmaxed, v)

torch.Size([3, 2, 16]) torch.Size([3, 2, 16, 8])


In [518]:
nh.shape

torch.Size([3, 2, 8])

In [519]:
ret = nh.reshape(batch_size, -1)

In [520]:
ret.shape

torch.Size([3, 16])

In [521]:
#softmaxed, 
ret = W_out(ret)

In [522]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        # calculate attention using function we will define next
        softmax, scores = attention(q, k, v, self.d_k, mask)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.d_model)
        
        output = softmax, self.out(concat)
    
        return output
def attention(q, k, v, d_k, mask=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  sqrt(d_k)
    scores = F.softmax(scores, dim=-1)


    output = torch.matmul(scores, v)
    return scores, output


In [523]:
target.shape

torch.Size([3, 16, 16])

In [524]:
query.shape

torch.Size([3, 16])

In [525]:
mh = MultiHeadAttention(2, 16)#heads, d_model

In [526]:
mh.q_linear = W_q
mh.v_linear = W_v
mh.k_linear = W_k
mh.out = W_out

In [527]:
sft, rettt = mh(query.unsqueeze(1), target, target)

In [528]:
sft.squeeze(2)

tensor([[[0.0639, 0.0597, 0.0631, 0.0625, 0.0682, 0.0628, 0.0629, 0.0593,
          0.0633, 0.0615, 0.0602, 0.0585, 0.0604, 0.0649, 0.0688, 0.0600],
         [0.0620, 0.0652, 0.0615, 0.0619, 0.0623, 0.0657, 0.0636, 0.0605,
          0.0616, 0.0636, 0.0597, 0.0607, 0.0611, 0.0629, 0.0628, 0.0648]],

        [[0.0589, 0.0597, 0.0622, 0.0599, 0.0631, 0.0643, 0.0632, 0.0668,
          0.0632, 0.0658, 0.0581, 0.0626, 0.0617, 0.0631, 0.0664, 0.0612],
         [0.0659, 0.0713, 0.0625, 0.0618, 0.0619, 0.0642, 0.0637, 0.0650,
          0.0588, 0.0643, 0.0589, 0.0614, 0.0655, 0.0571, 0.0626, 0.0550]],

        [[0.0648, 0.0642, 0.0635, 0.0646, 0.0614, 0.0630, 0.0608, 0.0611,
          0.0632, 0.0610, 0.0651, 0.0625, 0.0593, 0.0646, 0.0603, 0.0608],
         [0.0624, 0.0608, 0.0649, 0.0616, 0.0611, 0.0608, 0.0589, 0.0611,
          0.0618, 0.0640, 0.0683, 0.0611, 0.0610, 0.0654, 0.0606, 0.0662]]],
       grad_fn=<SqueezeBackward1>)

In [529]:
softmaxed

tensor([[[0.0639, 0.0597, 0.0631, 0.0625, 0.0682, 0.0628, 0.0629, 0.0593,
          0.0633, 0.0615, 0.0602, 0.0585, 0.0604, 0.0649, 0.0688, 0.0600],
         [0.0620, 0.0652, 0.0615, 0.0619, 0.0623, 0.0657, 0.0636, 0.0605,
          0.0616, 0.0636, 0.0597, 0.0607, 0.0611, 0.0629, 0.0628, 0.0648]],

        [[0.0589, 0.0597, 0.0622, 0.0599, 0.0631, 0.0643, 0.0632, 0.0668,
          0.0632, 0.0658, 0.0581, 0.0626, 0.0617, 0.0631, 0.0664, 0.0612],
         [0.0659, 0.0713, 0.0625, 0.0618, 0.0619, 0.0642, 0.0637, 0.0650,
          0.0588, 0.0643, 0.0589, 0.0614, 0.0655, 0.0571, 0.0626, 0.0550]],

        [[0.0648, 0.0642, 0.0635, 0.0646, 0.0614, 0.0630, 0.0608, 0.0611,
          0.0632, 0.0610, 0.0651, 0.0625, 0.0593, 0.0646, 0.0603, 0.0608],
         [0.0624, 0.0608, 0.0649, 0.0616, 0.0611, 0.0608, 0.0589, 0.0611,
          0.0618, 0.0640, 0.0683, 0.0611, 0.0610, 0.0654, 0.0606, 0.0662]]],
       grad_fn=<SoftmaxBackward>)

In [530]:
sft.shape

torch.Size([3, 2, 1, 16])

In [531]:
rettt.shape, ret.shape

(torch.Size([3, 1, 16]), torch.Size([3, 16]))

In [532]:
torch.sum(torch.abs(rettt.squeeze(1) - ret))

tensor(0., grad_fn=<SumBackward0>)